In [19]:
from openfermion.chem import MolecularData
from openfermion import FermionOperator
from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev
from openfermionpyscf import run_pyscf
from hamiltonian.matrix_hamiltonian import get_matrix_form
import numpy as np


# Set parameters to make a simple molecule.
diatomic_bond_length = .7414  # in Angstrom
geometry = [
    ('H', (0., 0., 0.)), 
    ('H', (0., 0., diatomic_bond_length))
]
basis = '3-21G'
# basis = 'sto-3G'
multiplicity = 1
charge = 0
description = str(diatomic_bond_length)

molecule = MolecularData(geometry, basis, multiplicity,
                         charge, description)

# Initializing the openfermionpyscf software
run_scf = 1
run_mp2 = 0
run_cisd = 0
run_ccsd = 0
run_fci = 1
delete_input = True
delete_output = True

# Run pyscf.
molecule = run_pyscf(molecule,
                    run_scf=run_scf,
                    run_mp2=run_mp2,
                    run_cisd=run_cisd,
                    run_ccsd=run_ccsd,
                    run_fci=run_fci
)

In [23]:
# Obtaining the fermionic Hamiltonian
molecular_hamiltonian = molecule.get_molecular_hamiltonian()
fermionic_hamiltonian = get_fermion_operator(molecular_hamiltonian)

# Removing the nuclear repuslion 
fermionic_hamiltonian += -FermionOperator('', fermionic_hamiltonian.terms[()])

# Obtaiing the Jordan-Wigner Hamiltonian
qubit_hamiltonian = jordan_wigner(fermionic_hamiltonian)
# bravyi_kitaev(fermionic_hamiltonian)


12

In [4]:
from openfermion.chem import MolecularData
from openfermion import FermionOperator
from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev
from openfermionpyscf import run_pyscf
from hamiltonian.matrix_hamiltonian import get_matrix_form
import numpy as np


# Set parameters to make a simple molecule.
diatomic_bond_length = .7414
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., diatomic_bond_length))]
# basis = '3-21G'
basis = 'sto-3G'
multiplicity = 1
charge = 0
description = str(diatomic_bond_length)

molecule = MolecularData(geometry, basis, multiplicity,
                         charge, description)

# Initializing the openfermionpyscf software
run_scf = 1
run_mp2 = 0
run_cisd = 0
run_ccsd = 0
run_fci = 0
delete_input = True
delete_output = True

# Run pyscf.
molecule = run_pyscf(molecule,
                        run_scf=run_scf,
                        run_mp2=run_mp2,
                        run_cisd=run_cisd,
                        run_ccsd=run_ccsd,
                        run_fci=run_fci)

In [5]:


# Obtaining the fermionic Hamiltonian
molecular_hamiltonian = molecule.get_molecular_hamiltonian()
fermionic_hamiltonian = get_fermion_operator(molecular_hamiltonian)

# Removing the nuclear repuslion 
fermionic_hamiltonian += -FermionOperator('', fermionic_hamiltonian.terms[()])

# Obtaiing the Jordan-Wigner Hamiltonian
qubit_hamiltonian = jordan_wigner(fermionic_hamiltonian)
# bravyi_kitaev(fermionic_hamiltonian)



In [11]:
# We can actually optimize it using only one, two, three and four neirest neighbouring parties
N = qubit_hamiltonian.many_body_order()
H = get_matrix_form(N, qubit_hamiltonian)
eig_vals, eig_vecs = np.linalg.eigh(H)
eig_vals[0]
psi_G = np.zeros((N**2, 1))
psi_G[:,0] = eig_vecs[:,0]
psi_G_trans = np.transpose(psi_G)

In [21]:
rho = np.matmul(psi_G, psi_G_trans)
np.trace(np.matmul(rho, rho))


# mmmm, weird, the state is pure, what could be wrong?

1.0000000000000002